In [1]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

MODEL_NAME = "skt/kogpt2-base-v2"
MODEL_PATH = "./models/"
SEQ_LEN = 50
TOKENS_DICT = {
    'bos_token':'</s>',
    'eos_token':'</s>',
    'unk_token':'<unk>',
    'pad_token':'<pad>',
    'mask_token':'<mask>',
    'additional_special_tokens':['<context>', '<slogan>'],
}

tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens(TOKENS_DICT)
model.resize_token_embeddings(len(tokenizer))

print(tokenizer.special_tokens_map)


{'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'additional_special_tokens': "['<context>', '<slogan>']"}


In [2]:
# HuggingFace에서 top k와 top p로 함수 샘플링
import torch
import torch.nn.functional as F
from tqdm import trange


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):

    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # top-k의 마지막 토큰보다 확률이 낮은 모든 토큰을 제거
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # 임계값 이상의 누적 확률을 가진 토큰 제거
        sorted_indices_to_remove = cumulative_probs > top_p
        
        # 첫 번째 토큰도 임계값보다 높게 유지하려면 인덱스를 오른쪽으로 이동
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # 정렬된 텐서를 원래 인덱싱에 분산
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits


# HuggingFace에서 컨텍스트/슬로건 분리 작업에 맞게 조정됨
def sample_sequence(model, length, context, segments_tokens=None, num_samples=1, temperature=1, top_k=0, top_p=0.0, repetition_penalty=1.0,
                    device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context

    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            if segments_tokens != None:
              inputs['token_type_ids'] = torch.tensor(segments_tokens[:generated.shape[1]]).unsqueeze(0).repeat(num_samples, 1)


            outputs = model(**inputs)  # 참고: GPT-2/Transfo-XL/XLNet/CTRL(캐시된 숨겨진 상태)과 함께 '과거'를 사용할 수도 있음
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # CTRL의 반복 페널티(https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling:
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated


In [3]:
context = "내일금융, 청년들을 위한 신용카드"

context_tkn = tokenizer.additional_special_tokens_ids[0]
slogan_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(context)

segments = [slogan_tkn] * SEQ_LEN
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [slogan_tkn]
# input_ids = input_ids + [slogan_tkn]

epochs = [5, 10, 15, 20]
for epoch in epochs:
  model.load_state_dict(torch.load(MODEL_PATH+f'{epoch}epoch_model_weights.pth'))
  model.eval()

  # Generate 20 samples of max length 20
  generated = sample_sequence(model, length=20, context=input_ids, segments_tokens=segments, num_samples=20)

  print('\n\n--- Generated Slogans ---\n')

  for g in generated:
    slogan = tokenizer.decode(g.squeeze().tolist())
    slogan = slogan.split('</s>')[0].split('<slogan>')[1]
    print(slogan)  

100%|██████████| 20/20 [00:19<00:00,  1.05it/s]




--- Generated Slogans ---

 의아서 지구를 타고없는 눈은 집이 배부는 아니 
 경모, 믿고 쑥! 
 틀겐, 전에 내일러물라 
 볼 때, 함께 아니다, 특별한 모든 나라 
_트 끝 나는 받는 때 세상을_까 
 담으면 있다면, 결혼 인긴니까 탐험착! 
 손금 
 나나비수, 내 인 밖에는 전망면으로 작동한 곳 
 제구를 준비되는 카페 
 소리 그겨울 건 전입해도 일 몸이 안기는 그랜있게 
 물은 등급내고 수 없죠 
 힘쓰고 없는 
 갱봅니다, 영혼에서 치킨은, 시대를 되다 
 알 수 없는 나라 햄 
 여행상을 아니니다 
 맞 숲에서 묻지요 
 아답게 6년가로 
 인생사로 만나고 반드시! 
 진정한 일을 위한 새로운 게 존재할주셔 있습니다 
 현재날?, 상식을 맞춘 시간이 있다 


100%|██████████| 20/20 [00:19<00:00,  1.04it/s]




--- Generated Slogans ---

 그 날, 기쁨, 화약이 하니까 
 함께볼 씹어지기 인생링하는 쓰세요 
 꿈여행, 국가대표, 이야기 
 가장 이미 당신의 미래를 크기입니다 
 이너가 어디에 편해서 아니다? 디지털에 열린다 
 디저격 다르게, 직옥콤함을 허물다 
 대한민국에 함께 벼슬이 많아 되고율? 
 오늘은 응 원유로 
 집 구할수록 행복다 
 지켜주고 싶은 혈관 돌려주고, 영화되지말고 
 좋은 010은 무료등을 넘어을 삽니다 
 모두가 이상 1동자 위의고, 맞감? 
 바로, 모두의 힘 
 지금 가장 치명적인, 전상,닝 
 준비하는 커피 
 좋은 보험을 내 마음껏 하다 
 굿맨만의맘 앞에 몰아파에다 
 실산, 땅이 결국, 온다 
 할 것은 내가겐날 
 내일의 즐거움이 혁신하는 시간 한마리를 있다 


100%|██████████| 20/20 [00:18<00:00,  1.10it/s]




--- Generated Slogans ---

 답의 끝 다 네라 
 민 보고날? 
 우리아이 첫 첫맘대로 시작된다 
 한 아이가생활의 시작 
 새로운 기회 
 가장 나다운 이이, 이렇게 나나 길 라니다 
 올봄, 역시, 쉽게 뜨직감 
 올겨울보다 믿음톡 
 부딪 옴니다 
 반반인 부담이잖아라 
 크리스 나도신이 있는 나, 네이디하는 방식 
 다시 북방의 나를 
 대한민국 행복생활 
 다 힘을 믿니다 
 모두가 꿈꾸네 
 주문해도 지금거데이 어만 허락하세요 
 초록산업의 미래를 찾니다 
 대한민국 전통시장 
 건조한 가장 지적인 위하여 
 청년이 열을 명 한 길다 


100%|██████████| 20/20 [00:18<00:00,  1.10it/s]



--- Generated Slogans ---

 세상을 달리는 모든 곳이 아니라 가족과 가까워지 
 사람을 위한 이 쇼핑 
 세상의 모든 책, 그비는저 
 모두에게 가는 곳이 아니라 아니라 보는 예술이야 
 이제 디드한 어버버 브랜드 될 수 있는 혜택몰할다 
 나다는 말로구, 챗 
 나의 나를 위한 내일로 인생을대전 
 내 삶의 태양이 더 부드러<unk>시키는 갑니다 
 중소기업이 아트다 
 아이에게 맴버가 내 운명을 아우! 
 너랑, 우리에겐를 다르게 
 가는 게 아니라갑다 
 겨울이 해열 말이 될수록 방이 있습니다 
 청년을 최고가그룹은 
 모두에게 추억이 내게도 드라마를 만나다 
 한 번을 더하다 
 사람을 위한 카드의 행복 
 다 함께라면 한 사람의 모든 여행 
 행복은 
 직관적 금융 


In [5]:
context = "신한카드, 청년들을 위한 신용카드"

context_tkn = tokenizer.additional_special_tokens_ids[0]
slogan_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(context)

segments = [slogan_tkn] * SEQ_LEN
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [slogan_tkn]


epochs = [5, 10, 15, 20]
for epoch in epochs:
  model.load_state_dict(torch.load(MODEL_PATH+f'{epoch}epoch_model_weights.pth'))
  model.eval()

  # Generate 20 samples of max length 20
  generated = sample_sequence(model, length=20, context=input_ids, segments_tokens=segments, num_samples=20)

  print('\n\n--- Generated Slogans ---\n')

  for g in generated:
    slogan = tokenizer.decode(g.squeeze().tolist())
    slogan = slogan.split('</s>')[0].split('<slogan>')[1]
    print(slogan)  

100%|██████████| 20/20 [00:17<00:00,  1.13it/s]




--- Generated Slogans ---

 모두의 꿈을 준 그리다 
 다시. 행복한 나를 위한 올림픽이 무한입니다 
 이럼을기술 
 밥 놀랄 대한민국 
 당 가능성이 이길 되는 사람들의 곳으로 
 마셔독한 카말시면 20리고머! 늘어뻑생하세요 
 오먼세 가지고 우리사이자 
 당신에게 다른어서 이어춰서 무도 휴카프? 
 덜 수 없는 어린이 더되었죠 
 새로움이 일상을 지켜춤 
 영심이 적일의 추억입 
 편하게 
 하나 수 없는 세계그룹,란트 
 안에서 사랑하는 대한민국 모든 게 개인 자산템 
 기업의 봄, 추입니다!, 따뜻한 선물을 
 시대가 하나의 당신에게 
 함께 존중없는 꿈도 1등 희망에 
 울려 때, 곧 길에 미래을 담다 
 완벽한 안에서였야 길로 위해,시간 만들고, 독드 
 초등님의 결정을 다르니까 


100%|██████████| 20/20 [00:17<00:00,  1.14it/s]




--- Generated Slogans ---

 차라리 마천례수, 학교가 기부례입니다 
 자본시장을 멀리기할요!요? 
 똑똑하게 사는 게 바로, 휴대폰에서! 
 대한 청춘과 생각위 
 관리하세요 
 자본시장을 멀립니다 사람 행복을 만들어갑다 
 만약로 하는 일자리 
 우리는 쓸 수 소비? 우리는 할인? 
 뻔, 늘 나누는 사람이든해 
 눈에, 다가겠습니다 
 기업합니다, 스마트 향수 부담 게 곧 자신됩니다 
 청년여행, 네아담하면 쉬워 
 심는 쇠고기 소비, 즐거운 사람들을 위해 
 우리 모두처럼 
 지금까지 안전의 전쟁을 아니라 열린까요로 돌아오세요 
 신한금융그룹과 함께 
 잡끄러자의 자꾸자의 소비세요 
 속도를 제주 세상을 부르는 생활 
 케이프과틀로 행복하다 
 당신의 한 사람을 세정갑니다 사람을 만든 테어로 지금 


100%|██████████| 20/20 [00:16<00:00,  1.20it/s]




--- Generated Slogans ---

 신한금융그룹과 희망을 바꾸는 기업금융그룹 
 신한금융그룹과 함께 세상을 키우는 가장 좋은 물 
 퍼포털 유아 전통 
 신한카드, 금융 
 아직 힘이 되어보세요 
 디지털로리 악플을 낳금융그룹은 제로로 금융을 만나든다 
 신한금융그룹과 함께하는 대한민국을 짓다 
 어용은 시작됩니다 
 신한금융을 만나다 설 일상을 입다 
 투이가드로 당신이 기다리았다는 보험 맴사 있습니다 
 즐거움을 지우는 세상 
 우리가 기업이다, 대한민국을 당신, 보험의 
 신한금융업을 답을 찾다 
 신한금융그룹과 함께하는 신한금융그룹 
 중소기업을 다다 
 금융을 부르는 주문 
 소비 주유카드가 다녀 절달팩리일 렌마트 서로의 보험입니다 
 신한금융그룹은 척리스트 
 내 플레이카 
 신한금융그룹은 아름다운 일상을 찾아줍니다 


100%|██████████| 20/20 [00:16<00:00,  1.22it/s]



--- Generated Slogans ---

실을 더하다 
 나의 봄이 
 나답게 올바르게 씀 
 하나처럼 가볍게 일상과 가까워지도록 
 금융생활에 다인인 당신도 송석 
 나는 움직리스트가다 
 올카드엔 심 하겠 꽃하고 받인을 디자인에 퍼펙 해주세요 
 신한금융의 기준 
 엄마, 그 정성으로 쇼핑으로 벗어나 시작 
 카드, 생활의 사람상을 돌려받는 거서다 
 금융의 해외주식 하겠습니다 
 그 사람생활이 시작됩워진다 
 부른들의 일상을 모험다 
 여행, 그다., 그 곳 
 마다. 카드하이드로 생활을 당신의 성공을 찾아준다 
 지금 같이 
 금융트업의 스페셜리스트로 따져보면있다 
 스타 년 알라 
실을 넘어선 쇼핑 있게 
 다시, 집이, 계곡집배입니다 
